# Import our modules and torch


In [1]:
import create_vocab
import data_to_tensors
from train_class import TrainingModule
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import random 
import model_implementation


RuntimeError: Failed to import transformers.models.bert because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
No module named 'tensorflow.python.keras.engine.keras_tensor'

# Because their data looks like this: we have 3 main dictionaries:
* dictionary of all words in the sheets
* dictionary of all paths(they are already hashed and converted to digits)
* dictionary of targets(method names)


In [6]:
dict_path = 'data/CODES_emb/CODES_emb.dict.c2v'
word2idx, path2idx, target2idx, idx2target = create_vocab.create_vocab(dict_path)

* 1) data_iterator Returns 1024 batches each(the size can be changed when calling the function)
* 2) from below you can see how the data is accessed, then 3 vectors are fed to the model(sorry for such names, I think you will understand from the arguments of the model that the output is)
* 3) well, at the output we have code_vector and the second vector-probability distribution between all labels in target_vocab(dictionary of method names)

In [7]:
# path_for_train = 'data/functions_emb/functions_emb.train.c2v'
path_for_train = 'data/CODES_emb/CODES_emb.train.c2v'
train_dataset = data_to_tensors.TextDataset(path_for_train, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

# path_for_val = 'data/functions_emb/functions_emb.val.c2v'
path_for_val = 'data/CODES_emb/CODES_emb.val.c2v'
val_dataset = data_to_tensors.TextDataset(path_for_val, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)
# path_for_test = 'data/functions_emb/functions_emb.test.c2v'
path_for_test = 'data/CODES_emb/CODES_emb.test.c2v'
test_dataset = data_to_tensors.TextDataset(path_for_test, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

# Train

In [8]:
SEED = 1337
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Model choose

* if you want to use the original model: bert = False
* if you want to use Bert model: bert = True


In [10]:
bert = False

In [11]:
# In case of bert
bert_params = dict()
bert_params['num_attention_heads'] = 1
bert_params['num_transformer_layers'] = 1
bert_params['intermediate_size'] = 32

if bert:
    model = model_implementation.code2vec_model(values_vocab_size = len(word2idx), 
                             paths_vocab_size = len(path2idx), 
                             labels_num = len(target2idx), bert=bert, bert_params=bert_params)

In [12]:
# In case if not bert
if not bert:
    model = model_implementation.code2vec_model(values_vocab_size = len(word2idx), 
                                 paths_vocab_size = len(path2idx), 
                                 labels_num = len(target2idx))

In [13]:
N_EPOCHS = 40
LR = 3e-3
WD = 0.8e-5

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)
criterion = nn.CrossEntropyLoss()

train_class = TrainingModule(model, optimizer, criterion, train_loader, val_loader, test_loader, N_EPOCHS, idx2target)
list_train_loss, list_val_loss, list_train_precision, list_val_precision,list_train_recall, list_val_recall, list_train_f1, list_val_f1 = train_class.train(bert, 'CODES_emb')


Epoch 1: train loss - 1.42532, validation loss - 1.21798 xyxyxyxyxyxyxyxxy
	 Validation: precision - 0.46154, recall - 0.46154, f1_score - 0.46154
$$$$$
Elapsed time: 2.794
----------------------------------------------------------------------
Epoch 2: train loss - 1.12971, validation loss - 1.19127 xyxyxyxyxyxyxyxxy
	 Validation: precision - 0.36923, recall - 0.36923, f1_score - 0.36923
$$$$$
Elapsed time: 2.715
----------------------------------------------------------------------
Epoch 3: train loss - 1.09331, validation loss - 1.21041 xyxyxyxyxyxyxyxxy
	 Validation: precision - 0.44615, recall - 0.44615, f1_score - 0.44615
$$$$$
Elapsed time: 4.543
----------------------------------------------------------------------
Epoch 4: train loss - 1.11219, validation loss - 1.17079 xyxyxyxyxyxyxyxxy
	 Validation: precision - 0.41538, recall - 0.41538, f1_score - 0.41538
$$$$$
Elapsed time: 5.591
----------------------------------------------------------------------
Epoch 5: train loss - 1.

# Predict and show names!

Go to weights of our models in order to take a view on how they pridect names: https://drive.google.com/drive/u/2/folders/1Q5ixv8dQ_qYqHg6w4Ep_XNeCJYZE6Cl2

You need to:
* Visit the above Drive folder.
* Right click on folder name -> "Add a shortcut to Drive" -> select "My Drive" -> "Add shortcut".

### Have an access to these weights

In [ ]:
# Detect if we are in Google Colaboratory
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
# Determine the locations of auxiliary libraries and datasets.
# `AUX_DATA_ROOT` is where 'notmnist.py', 'animation.py' and 'tiny-imagenet-2020.zip' are.
if IN_COLAB:
    google.colab.drive.mount("/content/drive/")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/My Drive/best_models/")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
else:
    AUX_DATA_ROOT = Path(".")

#### DEFINE model
* If you want to test original paper model:

In [ ]:
if bert == False:
  state_dict = torch.load('/Users/alishbahiqbal/Desktop/c2v_pytorch/Code2vec_Pytorch_pipeline/a3_q1_emb_article_model.pth')
else:
  state_dict = torch.load('/Users/alishbahiqbal/Desktop/c2v_pytorch/Code2vec_Pytorch_pipeline/CODES_emb_bert_model.pth')

model.load_state_dict(state_dict)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = model.to(DEVICE)

#### Let's test it on the test dataset

#  \<UNK\> means that this name doen't exist in our dictionary of labels:(

In [ ]:
import pandas as pd
from IPython.display import display

d = {'Original names': [], 'Predicted names': []}

for start, path, end, label in iter(test_loader):
    # get from model
    code, y_pred = model(start.to(DEVICE), path.to(DEVICE), end.to(DEVICE))
    # get probability
    y_pred = F.softmax(y_pred)
    # get best name index
    y_pred = torch.argmax(y_pred, dim = 1)
    
    for i, j in zip(label, y_pred):
      d['Original names'].append(idx2target[i.item()])
      d['Predicted names'].append(idx2target[j.item()])
    break

df = pd.DataFrame(data=d)
display(df,)

/opt/anaconda3/envs/fyp/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


,Original names,Predicted names
0,<unk>,7
1,10,7
2,10,7
3,7,7
4,7,7
...,...,...
59,10,10
60,2,7
61,7,7
62,10,7


In [ ]:
with pd.option_context('display.max_rows', 66, 'display.max_columns', 66):  # more options can be specified also
    print(df)

   Original names Predicted names
0           <unk>               7
1              10               7
2              10               7
3               7               7
4               7               7
5              10               7
6               7              10
7              10              10
8              10               7
9               7               7
10             10               7
11              5               7
12             10               7
13              7               7
14             10               7
15              7               7
16             10               7
17             10               7
18             10               7
19              7               7
20              7               7
21             10               7
22              5               7
23             10               7
24             10               7
25              7               7
26              7               7
27             10               7
28            